## Backup working notebook for data exploration

We will use awswrangler to make sure that we have a working connection to the AWS environment and that we can do our pandas-operations (groupby, melt etc.) without having to recreate the exact conditions

once this notebook executes successfullky, the next step is to migrate the code to `mcs-melter.py` which is in the root directory of this folder

### To-do
* migrate code to lambda
* exploration of more libraries being added for trajectory performance

In [1]:
# import required libraries

import boto3
import awswrangler as wr
import pandas as pd

In [2]:
# create boto session

session = boto3.session.Session(profile_name='saml')
s3 = session.resource('s3')

In [3]:
#using S3 objects to generate file list
# my_bucket = s3.Bucket('xwifi-od-s3-transform')
# data_prefix='/converted-data/snmp/type=radioStatsMCS/'
# file_list = [obs for obs in my_bucket.objects.all()]


In [4]:
# file_list

In [5]:
# file_list = wr.s3.list_objects("s3://xwifi-od-s3-transform/radioStatsMCS")

In [6]:
# # identify csv bucket url

# # muy_bucket = 'xwifi-od-s3-transform/'
# my_bucket = 's3://xwifi-od-s3-transform/converted-data/snmp/type=radioStatsMCS/**/*'

# # store results in a list
# result = wr.s3.list_objects(my_bucket, boto3_session=session)

In [7]:
# len(result)

In [8]:
# result[0]

In [9]:
# reading parquet files

parquet_bucket = 's3://xwifi-od-s3-parquet/snmp/type=radiostatsMCS/*'
result = wr.s3.list_objects(parquet_bucket, boto3_session=session)

In [10]:
# len(result)

In [11]:
# result[0]

In [12]:
df = wr.s3.read_parquet(result[0], boto3_session=session)

In [13]:
# df.head()

In [14]:
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date
df = df.drop(['current_ip', 'current_location', 'dt'], axis=1)
df = df.rename(columns={'current_mac':'snmpreportedmac',
                       'timestamp':'date'})
df = df.fillna(0)

In [16]:
# df.columns

In [15]:
column_dict = {}
for idx, c in enumerate(df.columns):
    if idx <=1:
        continue
    else:
        column_dict[c]= int

In [17]:
# column_dict

In [18]:
# Dataframe operations


df = df.astype(column_dict)


In [19]:
# df.head()

In [20]:
# Create the aggregate dictionary
agg_dict = {}
for idx, c in enumerate(df.columns):
    if idx <=1:
        continue
    else:
        agg_dict[c]= sum

In [21]:
# Basic aggregation & melting

df = df.groupby(['date','snmpreportedmac'], as_index=False).agg(agg_dict)
column_list = tuple(var for var in df.columns if var not in ['date', 'snmpreportedmac'])
df = pd.melt(df, id_vars=['date', 'snmpreportedmac'], value_vars=column_list) 

In [22]:
# df.head()

In [23]:
# regex to create sort_order, tranmission_type & radio_type columns

df['sort_order'] = df['variable'].str.extract(r'[2,5]\.?[4]?_\w{2}_mcs(\d*)')
df['transmission_type'] = df['variable'].str.extract(r'[2,5]\.?[4]?_(\w{2})_mcs\d*')
df['radio_type'] = df['variable'].str.extract(r'([2,5]\.?[4]?)_\w{2}_mcs\d*')

In [26]:
# df.head(20)